# Tutorial: Modelo de Predicción de Mora con PCA

## Introducción

Este notebook demuestra el proceso de construcción de un modelo de predicción para la variable 'Mora' (incumplimiento de pago). Un aspecto clave de este tutorial es el uso del Análisis de Componentes Principales (PCA) para la reducción de dimensionalidad antes de entrenar un modelo RandomForestClassifier. Además, el notebook ilustra un proceso iterativo de modelado, donde se identifica y corrige un problema común de fuga de datos (data leakage), lo cual es una lección importante en la práctica de la ciencia de datos.

**Pasos del Tutorial:**
1.  Configuración inicial y carga de datos.
2.  Preparación de datos y selección inicial de características.
3.  Primer intento de modelado con PCA, que revelará un problema de fuga de datos.
4.  Segundo intento de modelado, corrigiendo la fuga de datos y aplicando PCA nuevamente.
5.  Estimación de la importancia de las características originales después de la aplicación de PCA.

A través de estos pasos, veremos cómo se puede refinar un modelo y la importancia de una cuidadosa selección y preparación de características.

## 1. Configuración Inicial y Carga de Datos

### 1.1 Importación de Librerías Necesarias
Comenzamos importando las librerías que se utilizarán para la manipulación de datos, visualización, preprocesamiento y modelado.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### 1.2 Carga del Dataset
Descargamos y cargamos el dataset `df_analisis.parquet`, que contiene los datos preprocesados de facturación y clientes de un notebook anterior.

In [ ]:
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/df_analisis.parquet
df_analisis = pd.read_parquet('df_analisis.parquet')

## 2. Preparación de Datos para la Predicción

El título original de esta sección era "# Exploración de los datos", pero dado el contenido, se enfoca más en la preparación para la predicción.

In [ ]:
# df_analisis = pd.read_parquet('df_analisis.parquet') # Ya cargado arriba
# Visualización inicial para recordar la estructura
print("--- Primeras filas de df_analisis ---")
print(df_analisis.head())
df_analisis.info()

### 2.1 Selección Inicial de Características
El notebook original titula esta sección "# Ejercicios de prediccion con PCA".
Se selecciona un subconjunto de columnas de `df_analisis` para crear `df_prediccion`. Es importante notar que en esta selección inicial se incluye `Dias_PagoOportuno_PagoReal`, una variable que está directamente relacionada con la definición de `Mora`.

In [ ]:
df_prediccion = df_analisis[['Numero de factura', 'Consumo (m3)', 'Estrato', 'Precio por Consumo', 'Dias_Emision_PagoOportuno', 'Dias_Lectura_Emision', 'Dias_PagoOportuno_PagoReal', 'Mora']]
print("--- df_prediccion (primeras filas después de selección inicial) ---")
print(df_prediccion.head())

### 2.2 Preprocesamiento Básico de Características
A continuación, se realizan varias tareas de preprocesamiento:
- Se elimina la columna 'Numero de factura', ya que es un identificador y no una característica predictiva.
- La columna 'Estrato' se convierte de un formato de texto (ej. "Estrato 1") a un valor numérico entero.
- Los valores faltantes (NaN) en el DataFrame se rellenan con la media de sus respectivas columnas.

In [ ]:
df_prediccion_ml = df_prediccion.drop('Numero de factura', axis=1).copy() # Se usa .copy() para evitar SettingWithCopyWarning

# Convertir 'Estrato' a numérico ordinal
if df_prediccion_ml['Estrato'].dtype == 'object' or isinstance(df_prediccion_ml['Estrato'].dtype, pd.CategoricalDtype):
    df_prediccion_ml['Estrato'] = df_prediccion_ml['Estrato'].str.replace('Estrato ', '', regex=False).astype(int)
else:
    df_prediccion_ml['Estrato'] = df_prediccion_ml['Estrato'].astype(int)

# Imputar NaNs con la media de cada columna
for col in df_prediccion_ml.columns:
    if df_prediccion_ml[col].isnull().any():
        df_prediccion_ml[col].fillna(df_prediccion_ml[col].mean(), inplace=True)

print("--- df_prediccion_ml después de preprocesamiento básico (primeras filas) ---")
print(df_prediccion_ml.head())
print("\n--- Verificación de NaNs restantes ---")
print(df_prediccion_ml.isnull().sum())

## 3. Modelado con PCA - Primer Intento (Ilustrando Fuga de Datos)

En este primer intento, construiremos un modelo utilizando las características preparadas, incluyendo `Dias_PagoOportuno_PagoReal`. Como se mencionó, esto probablemente resultará en una fuga de datos.

### 3.1 Definición de Características (X) y Variable Objetivo (y)

In [ ]:
X_leak = df_prediccion_ml.drop('Mora', axis=1)
y_leak = df_prediccion_ml['Mora']

print("--- Características X_leak (primeras filas) ---")
print(X_leak.head())
print("\n--- Variable objetivo y_leak (primeras filas) ---")
print(y_leak.head())

### 3.2 Escalado de Características
Antes de aplicar PCA, es crucial escalar las características para que todas tengan una media de 0 y una desviación estándar de 1. Esto asegura que las características con magnitudes mayores no dominen el análisis de componentes principales.

In [ ]:
scaler_leak = StandardScaler()
X_scaled_leak = scaler_leak.fit_transform(X_leak)

print("--- X_scaled_leak (primeras 5 filas de características escaladas) ---")
print(pd.DataFrame(X_scaled_leak, columns=X_leak.columns).head())

### 3.3 Aplicación de PCA
Aplicamos PCA para reducir la dimensionalidad del conjunto de datos. Se configura `n_components=0.95`, lo que significa que PCA seleccionará el número mínimo de componentes principales que logren explicar al menos el 95% de la varianza en los datos escalados.

In [ ]:
pca_leak = PCA(n_components=0.95)
X_pca_leak = pca_leak.fit_transform(X_scaled_leak)

print(f"Número de componentes seleccionados por PCA (con leakage): {pca_leak.n_components_}")
print(f"Varianza explicada por cada componente: {pca_leak.explained_variance_ratio_}")
print(f"Varianza explicada acumulada: {pca_leak.explained_variance_ratio_.sum():.4f}")
print("\n--- X_pca_leak (primeras 5 filas de componentes principales) ---")
print(pd.DataFrame(X_pca_leak).head())

### 3.4 División en Conjuntos de Entrenamiento y Prueba

In [ ]:
X_train_pca_leak, X_test_pca_leak, y_train_leak, y_test_leak = train_test_split(
    X_pca_leak, y_leak, test_size=0.2, random_state=42, stratify=y_leak
)

print(f"Dimensiones de X_train_pca_leak: {X_train_pca_leak.shape}")
print(f"Dimensiones de X_test_pca_leak: {X_test_pca_leak.shape}")

### 3.5 Entrenamiento y Evaluación del Modelo RandomForest (Intento 1)
Se entrena un RandomForestClassifier utilizando los componentes principales. El parámetro `class_weight='balanced'` se usa para manejar el posible desbalance de clases en la variable 'Mora'.

In [ ]:
model_leak = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_leak.fit(X_train_pca_leak, y_train_leak)

y_pred_leak = model_leak.predict(X_test_pca_leak)

print("--- Evaluación del Modelo (Intento 1 - con posible leakage) ---")
print(f"Accuracy: {accuracy_score(y_test_leak, y_pred_leak):.4f}")
print("\nClassification Report:\n", classification_report(y_test_leak, y_pred_leak))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_leak, y_pred_leak))

**Análisis del Intento 1:**
Como se puede observar en la salida (que el notebook original generó con una accuracy de 1.0 y un reporte de clasificación perfecto), el modelo parece predecir la 'Mora' perfectamente. Esto es una fuerte indicación de **fuga de datos**, causada por la inclusión de la variable `Dias_PagoOportuno_PagoReal` que está directamente relacionada con cómo se calcula la 'Mora'.

# Segunda prueba sin la variable Dias_PagoOportuno_PagoReal

## 4. Modelado con PCA - Segundo Intento (Corrigiendo Fuga de Datos)

En este segundo intento, eliminaremos la variable `Dias_PagoOportuno_PagoReal` para construir un modelo más robusto y realista.

### 4.1 Preparación de Datos Sin Fuga de Datos

In [ ]:
# Reutilizamos df_prediccion_ml que ya tiene 'Estrato' convertido y NaNs imputados,
# pero esta vez eliminamos 'Dias_PagoOportuno_PagoReal'
df_prediccion_ml_sin_leak = df_prediccion_ml.drop('Dias_PagoOportuno_PagoReal', axis=1)

X_sin_leak = df_prediccion_ml_sin_leak.drop('Mora', axis=1)
y_sin_leak = df_prediccion_ml_sin_leak['Mora']

print("--- Características X_sin_leak (primeras filas) ---")
print(X_sin_leak.head())

### 4.2 Escalado de Características y Aplicación de PCA (Sin Fuga)

In [ ]:
scaler_sin_leak = StandardScaler()
X_scaled_sin_leak = scaler_sin_leak.fit_transform(X_sin_leak)

pca_sin_leak = PCA(n_components=0.95)
X_pca_sin_leak = pca_sin_leak.fit_transform(X_scaled_sin_leak)

print(f"Número de componentes seleccionados por PCA (sin leakage): {pca_sin_leak.n_components_}")
print(f"Varianza explicada acumulada (sin leakage): {pca_sin_leak.explained_variance_ratio_.sum():.4f}")
print("\n--- X_pca_sin_leak (primeras 5 filas de componentes principales) ---")
print(pd.DataFrame(X_pca_sin_leak).head())

### 4.3 División en Conjuntos de Entrenamiento y Prueba (Sin Fuga)

In [ ]:
X_train_pca_sin_leak, X_test_pca_sin_leak, y_train_sin_leak, y_test_sin_leak = train_test_split(
    X_pca_sin_leak, y_sin_leak, test_size=0.2, random_state=42, stratify=y_sin_leak
)

print(f"Dimensiones de X_train_pca_sin_leak: {X_train_pca_sin_leak.shape}")
print(f"Dimensiones de X_test_pca_sin_leak: {X_test_pca_sin_leak.shape}")

### 4.4 Entrenamiento y Evaluación del Modelo RandomForest (Intento 2 - Sin Fuga)

In [ ]:
model_sin_leak = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_sin_leak.fit(X_train_pca_sin_leak, y_train_sin_leak)

y_pred_sin_leak = model_sin_leak.predict(X_test_pca_sin_leak)

print("--- Evaluación del Modelo (Intento 2 - sin leakage) ---")
print(f"Accuracy: {accuracy_score(y_test_sin_leak, y_pred_sin_leak):.4f}")
print("\nClassification Report:\n", classification_report(y_test_sin_leak, y_pred_sin_leak))

cm_sin_leak = confusion_matrix(y_test_sin_leak, y_pred_sin_leak)
print("\nConfusion Matrix:\n", cm_sin_leak)

# Visualización de la Matriz de Confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm_sin_leak, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Mora (0)', 'Mora (1)'], yticklabels=['No Mora (0)', 'Mora (1)'])
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión (Modelo sin Fuga de Datos y con PCA)')
plt.show()

**Análisis del Intento 2:**
Al eliminar la variable que causaba la fuga de datos, el modelo ahora produce métricas de rendimiento más realistas. El accuracy y otras métricas (precision, recall, F1-score) reflejan mejor la capacidad del modelo para generalizar a datos no vistos.

## 5. Estimación de la Importancia de las Características Originales Después de PCA

Dado que el modelo RandomForest se entrenó con los componentes principales, no obtenemos directamente la importancia de las características originales. Sin embargo, podemos estimarla examinando cómo cada característica original contribuye a los componentes principales y cómo estos componentes, a su vez, son importantes para el modelo.

In [ ]:
# Importancia de los componentes principales según el modelo RandomForest
pca_component_importances = model_sin_leak.feature_importances_

# Loadings: indican la contribución de cada característica original a cada componente principal.
# pca_sin_leak.components_ tiene forma (n_components, n_original_features)
original_feature_loadings_abs = np.abs(pca_sin_leak.components_)

# Ponderar la contribución de cada característica a un componente por la importancia de ese componente
weighted_loadings = original_feature_loadings_abs * pca_component_importances[:, np.newaxis]

# Sumar las contribuciones ponderadas a través de todos los componentes para cada característica original
estimated_original_feature_importances = weighted_loadings.sum(axis=0)

# Normalizar para que la suma total sea 1 (opcional, para facilitar la comparación)
estimated_original_feature_importances_normalized = estimated_original_feature_importances / estimated_original_feature_importances.sum()

feature_names_sin_leak = X_sin_leak.columns
importance_df = pd.DataFrame({
    'Feature': feature_names_sin_leak,
    'Estimated_Importance': estimated_original_feature_importances_normalized
}).sort_values(by='Estimated_Importance', ascending=False)

print("--- Importancia Estimada de Características Originales (Post-PCA) ---")
print(importance_df)

plt.figure(figsize=(10, 7))
sns.barplot(x='Estimated_Importance', y='Feature', data=importance_df, palette='viridis')
plt.title('Importancia Estimada de Características Originales (Post-PCA)')
plt.show()

**Interpretación de la Importancia Estimada:**
Este análisis nos proporciona una visión de qué características originales tienen mayor influencia en la predicción del modelo, incluso después de la transformación PCA. Las características con una "Importancia Estimada" más alta son aquellas que contribuyen significativamente a los componentes principales que el modelo RandomForest consideró importantes.

## 6. Conclusión del Tutorial

En este tutorial, hemos cubierto un ciclo de modelado predictivo que incluyó:
- Preparación de datos y codificación de características.
- Un primer intento de modelado que nos permitió identificar y entender el concepto de fuga de datos.
- La corrección de la fuga de datos para un segundo intento de modelado más realista.
- La aplicación de PCA para reducción de dimensionalidad, explicando cómo se seleccionan los componentes (basado en la varianza explicada).
- El entrenamiento y evaluación de un RandomForestClassifier sobre los datos transformados por PCA.
- Un método para estimar la importancia de las características originales después de aplicar PCA.

**Aprendizajes Clave:**
- La **vigilancia contra la fuga de datos** es crucial. Resultados "demasiado buenos para ser verdad" a menudo indican este problema.
- **PCA puede ser una herramienta útil** para reducir la dimensionalidad, lo que puede ayudar a simplificar modelos, reducir el tiempo de entrenamiento y, en algunos casos, mejorar el rendimiento al eliminar ruido. Sin embargo, su efectividad debe evaluarse caso por caso.
- La **interpretabilidad del modelo** se vuelve más compleja con PCA, pero existen técnicas para estimar la influencia de las características originales.

**Próximos Pasos Posibles:**
- **Optimización de Hiperparámetros:** Ajustar los hiperparámetros tanto del RandomForestClassifier como de PCA (ej. el número de componentes o el umbral de varianza).
- **Comparación:** Entrenar el mismo modelo sin PCA (solo con datos escalados) y comparar el rendimiento y el tiempo de entrenamiento para evaluar el impacto real de PCA en este problema.
- **Explorar otros Modelos:** Probar diferentes algoritmos de clasificación.